# Data Collection_TradingView_Cryptocurrency

*20th, January, 2022*

In [1]:
'''import all necessary packages'''

from bs4 import BeautifulSoup
import requests as rq
import pandas as pd

Introduction: 

TradingView is a charting platform and social network used by 30M+ traders and investors launched in September 2011 by a group of traders and software developers. Users can use it to view price charts for a variety of different financial markets and asset classes, including stocks, currency rates, bonds, futures, and cryptocurrencies.

### Web scraping: Tradingview
https://www.tradingview.com/markets/cryptocurrencies/prices-all/

In [2]:
url = 'https://www.tradingview.com/markets/cryptocurrencies/prices-all/'
res = rq.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [3]:
'''make data structured and only take top 100 cryptocurrencies based on market_capitalization'''

coin = []
market_capitalization = []
fully_diluted_market_cap = []
last = []
available_coins = []
total_coins = []
traded_volume = []

count = 0
for i in soup.find_all('tr',{'class':'tv-data-table__row tv-data-table__stroke tv-screener-table__result-row'}):
    if count < 100: 
        coin.append(i.find('a').text)
        market_capitalization.append(i.find_all('td',{'class':'tv-data-table__cell tv-screener-table__cell tv-screener-table__cell--big'})[0].text)
        fully_diluted_market_cap.append(i.find_all('td',{'class':'tv-data-table__cell tv-screener-table__cell tv-screener-table__cell--big'})[1].text)
        last.append(i.find_all('td',{'class':'tv-data-table__cell tv-screener-table__cell tv-screener-table__cell--big'})[2].text)
        available_coins.append(i.find_all('td',{'class':'tv-data-table__cell tv-screener-table__cell tv-screener-table__cell--big'})[3].text)
        total_coins.append(i.find_all('td',{'class':'tv-data-table__cell tv-screener-table__cell tv-screener-table__cell--big'})[4].text)
        traded_volume.append(i.find_all('td',{'class':'tv-data-table__cell tv-screener-table__cell tv-screener-table__cell--big'})[5].text)
        count += 1

In [4]:
'''turn the data into DataFrame'''

crypto_market = {'coin':coin, 'market_capitalization':market_capitalization, 
                 'fully_diluted_market_cap':fully_diluted_market_cap, 
                 'traded_volume':traded_volume, 'last':last, 
                 'available_coins':available_coins, 'total_coins':total_coins}
crypto_market = pd.DataFrame(crypto_market)
print(crypto_market.shape)
crypto_market.head()

(100, 7)


,coin,market_capitalization,fully_diluted_market_cap,traded_volume,last,available_coins,total_coins
0,Bitcoin,780.797B,865.927B,17.249B,41234.63,18.935M,21.000M
1,Ethereum,364.628B,364.628B,9.098B,3057.88,119.242M,119.242M
2,Tether,78.305B,82.038B,33.125B,1.00,78.325B,82.059B
3,Binance Coin,73.791B,73.791B,1.744B,446.90,165.117M,165.117M
4,USD Coin,46.408B,46.408B,2.633B,1.00,46.408B,46.408B


In [5]:
'''standardize statistical units, all in one dollar'''
def stand_unit(number_list):
    number = []
    for i in number_list:
        if 'B' in i:
            i = float(i.replace('B',''))*1000000000
        elif 'M' in i:
            i = float(i.replace('M',''))*1000000
        elif 'K' in i:
            i = float(i.replace('K',''))*1000
        else:
            i = float(i)
        number.append(i)
    return number
#     market_capitalization.append(i_ny)

In [6]:
crypto_market.market_capitalization = stand_unit(crypto_market.market_capitalization)

crypto_market.fully_diluted_market_cap = stand_unit(crypto_market.fully_diluted_market_cap)

crypto_market.traded_volume = stand_unit(crypto_market.traded_volume)

last = []
for i in crypto_market['last']:
    last.append(float(i))    
crypto_market['last'] = last

crypto_market.available_coins = stand_unit(crypto_market.available_coins)

crypto_market.total_coins = stand_unit(crypto_market.total_coins)

crypto_market.head()

,coin,market_capitalization,fully_diluted_market_cap,traded_volume,last,available_coins,total_coins
0,Bitcoin,7.807970e+11,8.659270e+11,1.724900e+10,41234.63,1.893500e+07,2.100000e+07
1,Ethereum,3.646280e+11,3.646280e+11,9.098000e+09,3057.88,1.192420e+08,1.192420e+08
2,Tether,7.830500e+10,8.203800e+10,3.312500e+10,1.00,7.832500e+10,8.205900e+10
3,Binance Coin,7.379100e+10,7.379100e+10,1.744000e+09,446.90,1.651170e+08,1.651170e+08
4,USD Coin,4.640800e+10,4.640800e+10,2.633000e+09,1.00,4.640800e+10,4.640800e+10


In [7]:
'''make the list coin as index for combinating other DataFrame later'''

crypto_market = crypto_market.set_index(['coin'])
crypto_market.head()

,market_capitalization,fully_diluted_market_cap,traded_volume,last,available_coins,total_coins
coin,,,,,,
Bitcoin,7.807970e+11,8.659270e+11,1.724900e+10,41234.63,1.893500e+07,2.100000e+07
Ethereum,3.646280e+11,3.646280e+11,9.098000e+09,3057.88,1.192420e+08,1.192420e+08
Tether,7.830500e+10,8.203800e+10,3.312500e+10,1.00,7.832500e+10,8.205900e+10
Binance Coin,7.379100e+10,7.379100e+10,1.744000e+09,446.90,1.651170e+08,1.651170e+08
USD Coin,4.640800e+10,4.640800e+10,2.633000e+09,1.00,4.640800e+10,4.640800e+10


In [8]:
crypto_market.to_csv('crypto_market_17012022.csv',index = True)